<a href="https://colab.research.google.com/github/KolipakaRamesh/Models/blob/main/LSTM_Stock_Market_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub

In [ ]:
from google.colab import userdata

# Get the token from Colab's Secrets Manager
hf_token = userdata.get('HF_TOKEN')

# Login to HF Hub using the token
from huggingface_hub import login
login(token=hf_token)

In [ ]:
# Train Model
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def load_data(stock_symbol='AAPL', start='2015-01-01', end='2023-12-31'):
    df = yf.download(stock_symbol, start=start, end=end)
    return df[['Close']]

def preprocess_data(data, sequence_length=60):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i - sequence_length:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, y, scaler

def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

X, y, scaler = None, None, None  # global placeholders

def train_model(stock_symbol='AAPL'):
    global X, y, scaler
    data = load_data(stock_symbol)
    X, y, scaler = preprocess_data(data)

    model = build_model((X.shape[1], 1))
    model.fit(X, y, epochs=5, batch_size=32)
    return model

model = train_model('AAPL')

/tmp/ipython-input-4032979482.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock_symbol, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - loss: 0.0447
Epoch 2/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 6.1007e-04
Epoch 3/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 5.8225e-04
Epoch 4/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - loss: 6.0352e-04
Epoch 5/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 5.3073e-04


In [ ]:
# Save Model and Scaler Locally
model.save('lstm_model.keras')
import joblib
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
# Upload Files to Hugging Face Hub
from huggingface_hub import HfApi

api = HfApi()
repo_id = "RameshKolipaka/stock-predictor-model"  # Replace with your HF repo name

# Create repo if not exist (optional)
api.create_repo(repo_id, exist_ok=True)

# Upload files
api.upload_file(
    path_or_fileobj="lstm_model.keras",
    path_in_repo="lstm_model.keras",
    repo_id=repo_id,
    repo_type="model",
)

api.upload_file(
    path_or_fileobj="scaler.pkl",
    path_in_repo="scaler.pkl",
    repo_id=repo_id,
    repo_type="model",
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  lstm_model.keras                      : 100%|##########|  400kB /  400kB            

No files have been modified since last commit. Skipping to prevent empty commit.


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  scaler.pkl                            : 100%|##########|   719B /   719B            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/RameshKolipaka/stock-predictor-model/commit/7198140274c6ad69fbe89f477b019a8b5c596fe7', commit_message='Upload scaler.pkl with huggingface_hub', commit_description='', oid='7198140274c6ad69fbe89f477b019a8b5c596fe7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RameshKolipaka/stock-predictor-model', endpoint='https://huggingface.co', repo_type='model', repo_id='RameshKolipaka/stock-predictor-model'), pr_revision=None, pr_num=None)